In [73]:
# pillow exifread geopy datetime
from PIL import Image
import exifread
import reverse_geocoder as rg
from datetime import datetime
from geopy.point import Point
from geopy.geocoders import Nominatim

In [ ]:
def get_exif_data(image_path):
    geolocator = Nominatim(user_agent="South Korea")
    
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)
    # GPS 정보 추출
    latitude = tags.get('GPS GPSLatitude')
    longitude = tags.get('GPS GPSLongitude')
    timestamp = tags.get('EXIF DateTimeOriginal')
    
    
    if latitude and longitude:
        latitude_sec = float(latitude.values[2].num) / float(latitude.values[2].den)
        longitude_sec = float(longitude.values[2].num) / float(longitude.values[2].den)

        latitude_dms = f"{latitude.values[0]} {latitude.values[1]}m {latitude_sec}s N"
        longitude_dms = f"{longitude.values[0]} {longitude.values[1]}m {longitude_sec}s E"
        
        
        # location = "beep"
        point = Point(latitude_dms + " " + longitude_dms)
        
        # location = rg.search((point.latitude, point.longitude))
        location = geolocator.reverse((point.latitude, point.longitude))
        
        # 구글, 네이버, 카카오 등 시험해볼것
    else:
        location = None

    return {
        "location": location[:-1] if location else "위치 정보 없음",
        "timestamp": timestamp.printable if timestamp else "시간 정보 없음"
    }

In [92]:
paths = ["2025-01-21-20-07-25-356.jpg", "2714.jpg", "3131.JPG", "4066.JPG"]
for path in paths:
  print(get_exif_data(path))

{'location': '위치 정보 없음', 'timestamp': '시간 정보 없음'}
{'location': ('신촌 포가레, 26, 명물길, 대현동, 신촌동, 서대문구, 서울특별시, 03777, 대한민국',), 'timestamp': '2014:06:22 14:56:29'}
{'location': ('홍지문터널, 부암동, 평창동, 종로구, 서울특별시, 03018, 대한민국',), 'timestamp': '2014:08:10 12:33:58'}
{'location': ('반여로, 반여동, 반여1동, 해운대구, 부산광역시, 48038, 대한민국',), 'timestamp': '2015:01:18 14:37:05'}


In [90]:
# tensorflow
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [91]:
# ResNet50 모델 로드
model = ResNet50(weights='imagenet')

def predict_location(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    preds = model.predict(img_array)
    decoded_preds = decode_predictions(preds, top=3)[0]  # 상위 3개 후보 반환
    
    return [(label, prob) for (_, label, prob) in decoded_preds]

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [93]:
paths = ["2025-01-21-20-07-25-356.jpg", "2714.jpg", "3131.JPG", "4066.JPG"]
for path in paths:
  print(predict_location(path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[('tarantula', np.float32(0.16282938)), ('tray', np.float32(0.14837226)), ('matchstick', np.float32(0.10072878))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
[('punching_bag', np.float32(0.98774314)), ('scale', np.float32(0.003435158)), ('mailbox', np.float32(0.0030469736))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
[('maze', np.float32(0.25024658)), ('park_bench', np.float32(0.13781811)), ('stone_wall', np.float32(0.10852632))]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
[('home_theater', np.float32(0.779898)), ('screen', np.float32(0.05267543)), ('scuba_diver', np.float32(0.033569157))]


In [ ]:
# opencv-python, pytesseract
# conda install -c conda-forge tesseract

import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [98]:
def extract_text(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, lang='eng+kor')  # 영어+한글 OCR
    return text

In [101]:
for path in paths:
  print(extract_text(path))

TesseractNotFoundError: C:\Program Files\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.